In [2]:
import numpy as np


class LinearRegression:
    """
    Multi-purpose linear model supporting:
      - Regression (MSE) and binary Classification (sigmoid + BCE)
      - Multi-feature inputs (X shape: [n_samples, n_features])
      - Options: fit_intercept, normalize, batch_size (None => full-batch),
                 L2 regularization (penalty='l2', alpha=float)
      - Methods: fit(), predict(), predict_proba(), score()
    Note: Classification implemented is binary (0/1). For multiclass, use one-vs-rest externally.
    """

    def __init__(
        self,
        lr=0.01,
        n_iters=1000,
        task="regression",        # "regression" or "classification"
        batch_size=None,          # None -> full-batch, else integer for mini-batch
        fit_intercept=True,
        # whether to standardize X (z-score) before training
        normalize=False,
        penalty=None,             # None or 'l2'
        alpha=0.0,                # regularization strength (only for l2)
        threshold=0.5,            # classification threshold for predict()
    ):
        self.lr = lr
        self.n_iters = n_iters
        self.task = task
        self.batch_size = batch_size
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.penalty = penalty
        self.alpha = alpha
        self.threshold = threshold

        # model parameters (initialized in fit)
        self.weights = None   # shape (n_features,)
        self.bias = 0.0
        # normalization params (if requested)
        self._X_mean = None
        self._X_std = None

    # ---------------------------
    # Utility functions
    # ---------------------------
    def _prepare_X(self, X, training=False):
        """
        Ensure X is numpy array, optionally normalize using stored params (or compute them if training=True).
        """
        X = np.array(X, dtype=float)
        if self.normalize:
            if training:
                self._X_mean = X.mean(axis=0)
                self._X_std = X.std(axis=0)
                # avoid division by zero
                self._X_std[self._X_std == 0.0] = 1.0
            X = (X - self._X_mean) / self._X_std
        return X

    @staticmethod
    def _sigmoid(z):
        # Numerically stable sigmoid
        z = np.clip(z, -500, 500)
        return 1.0 / (1.0 + np.exp(-z))

    # ---------------------------
    # Training
    # ---------------------------
    def fit(self, X, y):
        """
        Train the model.
          - X: array-like shape (n_samples, n_features)
          - y: array-like shape (n_samples,)  for regression: numeric, for classification: {0,1}
        """
        X = self._prepare_X(X, training=True)
        y = np.array(y, dtype=float)

        n_samples, n_features = X.shape

        # initialize weights and bias
        self.weights = np.zeros(n_features, dtype=float)
        # bias kept even if fit_intercept False (not used if False)
        self.bias = 0.0 if self.fit_intercept else 0.0

        # mini-batch logic
        if self.batch_size is None:
            batch_size = n_samples
        else:
            batch_size = int(self.batch_size)
            batch_size = max(1, min(batch_size, n_samples))

        for it in range(self.n_iters):
            # shuffle indices for each epoch if using mini-batches
            indices = np.arange(n_samples)
            if batch_size < n_samples:
                np.random.shuffle(indices)

            for start in range(0, n_samples, batch_size):
                batch_idx = indices[start:start + batch_size]
                X_b = X[batch_idx]
                y_b = y[batch_idx]
                m = X_b.shape[0]  # current batch size

                # predictions
                linear_output = X_b.dot(
                    self.weights) + (self.bias if self.fit_intercept else 0.0)

                if self.task == "regression":
                    # Mean Squared Error gradient:
                    # loss = (1/m) * sum((pred - y)^2)
                    # dL/dw = (2/m) * X^T (pred - y)
                    error = linear_output - y_b
                    dw = (2.0 / m) * X_b.T.dot(error)
                    db = (2.0 / m) * np.sum(error) if self.fit_intercept else 0.0

                    # L2 regularization (ridge)
                    if self.penalty == "l2" and self.alpha > 0:
                        dw += 2.0 * self.alpha * self.weights

                elif self.task == "classification":
                    # Binary cross-entropy:
                    # preds = sigmoid(linear_output)
                    preds = self._sigmoid(linear_output)
                    # gradient of BCE w.r.t. weights: (1/m) * X^T (preds - y)
                    error = preds - y_b
                    dw = (1.0 / m) * X_b.T.dot(error)
                    db = (1.0 / m) * np.sum(error) if self.fit_intercept else 0.0

                    if self.penalty == "l2" and self.alpha > 0:
                        dw += 2.0 * self.alpha * self.weights
                else:
                    raise ValueError(
                        "task must be 'regression' or 'classification'")

                # update parameters
                self.weights -= self.lr * dw
                if self.fit_intercept:
                    self.bias -= self.lr * db

        return self

    # ---------------------------
    # Prediction
    # ---------------------------
    def predict_proba(self, X):
        """
        For regression: returns continuous predictions (same as predict)
        For classification: returns probability of class 1 (sigmoid output)
        """
        X = self._prepare_X(X, training=False)
        linear_output = X.dot(self.weights) + \
            (self.bias if self.fit_intercept else 0.0)
        if self.task == "regression":
            return linear_output
        else:
            return self._sigmoid(linear_output)

    def predict(self, X):
        """
        For regression: continuous outputs
        For classification: returns binary labels (0/1) using threshold
        """
        proba_or_pred = self.predict_proba(X)
        if self.task == "regression":
            return proba_or_pred
        else:
            return (proba_or_pred >= self.threshold).astype(int)

    # ---------------------------
    # Scoring / metrics
    # ---------------------------
    def score(self, X, y, metric=None):
        """
        Compute performance score:
          - For regression (default): R^2 score (coefficient of determination)
              other metric choices: "mse", "mae", "r2"
          - For classification (default): accuracy
              other metric choices: "accuracy", "precision", "recall", "f1"
        """
        X = self._prepare_X(X, training=False)
        y = np.array(y)

        if self.task == "regression":
            y_pred = self.predict(X)
            if metric is None or metric == "r2":
                # R^2
                ss_res = np.sum((y - y_pred) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                # guard against divide by zero
                return 1.0 - ss_res / ss_tot if ss_tot != 0 else 0.0
            elif metric == "mse":
                return np.mean((y - y_pred) ** 2)
            elif metric == "mae":
                return np.mean(np.abs(y - y_pred))
            else:
                raise ValueError(
                    "Unsupported metric for regression. Choose 'r2', 'mse' or 'mae'.")

        else:  # classification
            y_pred = self.predict(X)
            if metric is None or metric == "accuracy":
                return np.mean(y_pred == y)
            elif metric == "precision":
                tp = np.sum((y_pred == 1) & (y == 1))
                fp = np.sum((y_pred == 1) & (y == 0))
                return tp / (tp + fp) if (tp + fp) > 0 else 0.0
            elif metric == "recall":
                tp = np.sum((y_pred == 1) & (y == 1))
                fn = np.sum((y_pred == 0) & (y == 1))
                return tp / (tp + fn) if (tp + fn) > 0 else 0.0
            elif metric == "f1":
                p = self.score(X, y, metric="precision")
                r = self.score(X, y, metric="recall")
                return 2 * p * r / (p + r) if (p + r) > 0 else 0.0
            else:
                raise ValueError(
                    "Unsupported metric for classification. Choose 'accuracy', 'precision', 'recall' or 'f1'.")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("./heart_statlog_cleveland_hungary_final.csv")

y = df["target"]
X = df.drop(columns=["target"])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train classification model
model = LinearRegression(lr=0.01, n_iters=2000, task="classification", normalize=True)
model.fit(X_train, y_train)

# Evaluate
print("Accuracy:", model.score(X_test, y_test, metric="accuracy"))
print("Precision:", model.score(X_test, y_test, metric="precision"))
print("Recall:", model.score(X_test, y_test, metric="recall"))
print("F1 Score:", model.score(X_test, y_test, metric="f1"))

# Predict probabilities
print("Probabilities:", model.predict_proba(X_test))

# Predict classes
print("Predicted classes:", model.predict(X_test))

Accuracy: 0.49328859060402686
Precision: 1.0
Recall: 0.10650887573964497
F1 Score: 0.0
Probabilities: [0.9166172  0.75649564 0.02456942 0.94855636 0.55713288 0.97706413
 0.1738604  0.36457849 0.76997118 0.3420673  0.27830342 0.39021214
 0.08668475 0.48908156 0.82463153 0.61204875 0.98421599 0.63882514
 0.06424871 0.97205305 0.92340993 0.87588039 0.60241156 0.76532619
 0.61101988 0.9377167  0.26244947 0.94316716 0.62818966 0.83340232
 0.73652749 0.72874281 0.24055403 0.98664635 0.95577443 0.41298055
 0.65846499 0.88427005 0.38063548 0.99327885 0.17931648 0.05804783
 0.43081355 0.05942756 0.86030348 0.85883804 0.93849992 0.92824058
 0.96051827 0.93744201 0.99101366 0.25672271 0.90781112 0.1066918
 0.94691775 0.91270345 0.11122039 0.06431108 0.15592645 0.9562983
 0.72952329 0.27987921 0.20266946 0.72598946 0.76490021 0.97381475
 0.03535259 0.91576644 0.89854197 0.59624142 0.08693864 0.3351853
 0.96458301 0.88044535 0.55386561 0.04776253 0.90822313 0.92803923
 0.84925964 0.85772294 0.05894